the task is to tune the RF model for each spot. there will be one set of hyperparameters used for the whole thing.
- i set spot 09 to be used as the test data. the accuracy and other necessary metrics will be determined from there.
- 


In [1]:
import pandas as pd
path = ''
df_01 = pd.read_csv(path+"df_01.csv")
df_02 = pd.read_csv(path+"df_02.csv")
df_03 = pd.read_csv(path+"df_03.csv")
df_04 = pd.read_csv(path+"df_04.csv")
df_05 = pd.read_csv(path+"df_05.csv")
df_06 = pd.read_csv(path+"df_06.csv")
df_07 = pd.read_csv(path+"df_07.csv")
df_09 = pd.read_csv(path+"df_09.csv")


In [2]:
train = pd.concat([df_01,df_02,df_03,df_04,df_05,df_06,df_07]);train=train.dropna(subset=['laf50_per_hour']) #drop NAs in the 'laf50_per_hour'
test=df_09
#keep in mind that operations done on train do not apply for the constituent dataframes.
#convert 'date' and 'day' to datetime for both train and test
train['date'] = pd.to_datetime(train["date"]);test['date'] = pd.to_datetime(test["date"])
train['day'] = pd.to_datetime(train["day"]);test['day'] = pd.to_datetime(test["day"])


i want to try using df_07 as validation, then tune hyperparameters with sklearn's GridSearchCV(). 
next i'll try to see if i can use nested CV: 7-fold hyperparameter tuning, in a sense.

In [3]:
train_X = train.drop(['date','day','time','noisiness','location','weekday'],axis=1)
train_Y = train['noisiness']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid = train_test_split(train_X, train_Y, test_size=0.3, random_state=19)
model = RandomForestClassifier()
model.fit(X_train,Y_train)


RandomForestClassifier()

In [ ]:
from sklearn.metrics import accuracy_score
#prediction checking
Y_pred = model.predict(X_valid)
accuracy = accuracy_score(Y_valid,Y_pred)
print(accuracy)

0.9971154595774538


99.7% is only done on validation data. 

In [ ]:
len(Y_test_pred)

6254

In [ ]:
#what if we tested on df_09 as planned? does it generalize?
test = test.dropna(subset=['laf50_per_hour'])
X_test = test.drop(['date','day','time','noisiness','location','weekday'],axis=1)
Y_test = test['noisiness']
Y_test_pred = model.predict(X_test)
accuracy_test = accuracy_score(Y_test_pred,Y_test)
print(accuracy_test)

0.7451231212024304


it generalizes a teensy bit better. now to try tuning, with RandomizedSearchCV() apparently being used before GridSearchCV().
source: https://www.geeksforgeeks.org/random-forest-hyperparameter-tuning-in-python/

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid = train_test_split(train_X, train_Y, test_size=0.3, random_state=19)
model = RandomForestClassifier()
param_grid_rs = {
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    'min_samples_leaf': [1, 4, 10, 50, 100],
    'min_samples_split': [2, 10, 20, 50],
    'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600]
}
rf_random = RandomizedSearchCV(estimator = model, param_distributions = param_grid_rs, cv = 5, verbose=2, random_state=19)
rf_random.fit(train_X, train_Y) #fit is to initiate training process


In [5]:
print(rf_random.best_estimator_)

RandomForestClassifier(max_depth=60, min_samples_leaf=4, n_estimators=600)


thus GridSearchCV() will be trained around max_depth = 60, min_samples_leaf=4, n_estimators=600.

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid_gs = {
    'max_depth': [50, 55, 60, 65, 70],
    'min_samples_leaf': [1, 4, 7, 10],
    'n_estimators': [400, 500, 600, 700, 800]
}
rf_grid = GridSearchCV(estimator = model, param_grid = param_grid_gs, cv = 5, verbose=2)
rf_grid.fit(train_X, train_Y) #fit is to initiate training process

In [13]:
print(rf_grid.best_estimator_)

RandomForestClassifier(max_depth=65, n_estimators=400)


min_samples_leaf apparently doesnt matter as much? but now the max_depth is 65 while n_estimators is 400. the hyperparams will be set to that.